In [2]:
!pip install selenium
!pip install bs4

In [6]:
!pip install chromedriver_binary

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for chromedriver_binary: filename=chromedriver_binary-122.0.6251.0.0-py3-none-any.whl size=8459642 sha256=e45e4cca97789bedb36dd250b9a75d4749b42d54f3b9d59288e752a36dbd66bc
  Stored in directory: c:\users\syed hasham hameed\appdata\local\pip\cache\wheels\e7\62\83\61108be18922da14b7a9d96d8b80a4c3f7c854fd2627748893
Successfully built chromedriver_binary


In [7]:
from selenium import webdriver 
import chromedriver_binary

In [93]:
driver = webdriver.Chrome()

In [95]:
url= 'https://www.amazon.com/'

In [96]:
driver.get(url)

In [97]:
def get_url(keyword):
    temp="https://www.amazon.com/s?k={}&ref=nb_sb_noss_2"
    keyword = keyword.replace(" ","+")
    temp = temp.format(keyword)
    return temp

In [98]:
url1 = get_url("laptop charger")
print(url1)

https://www.amazon.com/s?k=laptop+charger&ref=nb_sb_noss_2


In [99]:
driver.get(url1)

In [101]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(driver.page_source,"html.parser")

In [102]:
soup_results = soup.find_all("div",{"data-component-type":"s-search-result"})

In [103]:
len(soup_results)

20

In [104]:
obj = soup_results[0]

In [108]:
span_tag = obj.find("span","a-icon-alt").text

In [109]:
print(span_tag)

4.6 out of 5 stars


In [111]:
price = obj.find('span', {'class': 'a-size-base'}).text
print(price)

21


# FUNCTION TO GET EVERYTHING AT ONCE

In [92]:
def get_url(keyword):
    temp = "https://www.amazon.com/s?k={}&ref=nb_sb_noss_2"
    keyword = keyword.replace(" ","+")
    temp = temp.format(keyword)
    
    temp += "&page={}"
    
    return temp 

def extract(obj):
    
    h2_tag = obj.find('h2')
    
    if h2_tag:
        atag = h2_tag.find('a')
    else:
        None
        
    if atag:
        description = atag.get_text(strip=True)
    else:
        description = None 

    
    
    try:
        span_p = obj.find("span","a-price")
        price = span_p.find("span","a-offscreen").text
    
    except AttributeError:
        return
    
    try:
        ratings = obj.find("span","a-icon-alt").text
        reviews = obj.find('span', {'class': 'a-size-base'}).text
        
    except AttributeError:
        ratings = ""
        reviews = ""
    
    
    rec = (description, price, ratings, reviews)
    
    return rec


def main(keyword):
    import csv
    
    records = []
    
    driver = webdriver.Chrome()
    
    url = get_url(keyword)
    
    
    
    for page in range(1,25):
        
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source,"html.parser")
        soup_data = soup.find_all("div",{"data-component-type":"s-search-result"})
        
        for i in soup_data:
            measures = extract(i)
        
            if measures:
                records.append(measures)
                
            
    driver.quit()
            
    with open('Resultsscrap1.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'Reviews'])
        writer.writerows(records)
            
    
    

In [112]:
main("golf clubs")